In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1609600083865_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
database_name = "training"
bucket_name = "rate-my-post-2"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1609600083865_0016,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import defaultdict
import re

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# NLTK UDF
sia = SentimentIntensityAnalyzer()

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_sentences(df):
    return df.apply(lambda text: len(sent_tokenize(text)))

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_words(df):
    return df.apply(lambda text: len([w for w in word_tokenize(text) if re.search("[A-Za-z0-9]", w)]))

@f.pandas_udf(DoubleType(), f.PandasUDFType.SCALAR)
def get_sentiment_pos(df):
    return df.apply(lambda text: sia.polarity_scores(text)["pos"])

@f.pandas_udf(DoubleType(), f.PandasUDFType.SCALAR)
def get_sentiment_neg(df):
    return df.apply(lambda text: sia.polarity_scores(text)["neg"])

@f.pandas_udf(DoubleType(), f.PandasUDFType.SCALAR)
def get_sentiment_com(df):
    return df.apply(lambda text: sia.polarity_scores(text)["compound"])

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_nouns(df):
    return df.apply(lambda text: len([p for p in pos_tag(word_tokenize(text), tagset="universal")
                                      if p[1] == "NOUN" and re.search("[A-Za-z0-9]", p[0])]))

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_verbs(df):
    return df.apply(lambda text: len([p for p in pos_tag(word_tokenize(text), tagset="universal")
                                      if p[1] == "VERB" and re.search("[A-Za-z0-9]", p[0])]))

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_adjectives(df):
    return df.apply(lambda text: len([p for p in pos_tag(word_tokenize(text), tagset="universal")
                                      if p[1] == "ADJ" and re.search("[A-Za-z0-9]", p[0])]))

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_adverbs(df):
    return df.apply(lambda text: len([p for p in pos_tag(word_tokenize(text), tagset="universal")
                                      if p[1] == "ADV" and re.search("[A-Za-z0-9]", p[0])]))

@f.pandas_udf(IntegerType(), f.PandasUDFType.SCALAR)
def count_pronouns(df):
    return df.apply(lambda text: len([p for p in pos_tag(word_tokenize(text), tagset="universal")
                                      if p[1] == "PRON" and re.search("[A-Za-z0-9]", p[0])]))


spark.udf.register("udf_count_sentences", count_sentences)
spark.udf.register("udf_count_words", count_words)
spark.udf.register("udf_get_sentiment_pos", get_sentiment_pos)
spark.udf.register("udf_get_sentiment_neg", get_sentiment_neg)
spark.udf.register("udf_get_sentiment_com", get_sentiment_com)
spark.udf.register("udf_count_nouns", count_nouns)
spark.udf.register("udf_count_verbs", count_verbs)
spark.udf.register("udf_count_adjectives", count_adjectives)
spark.udf.register("udf_count_adverbs", count_adverbs)
spark.udf.register("udf_count_pronouns", count_pronouns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function count_pronouns at 0x7f5c3c5c25f0>

In [5]:
# spark.sql(f"""
# drop database if exists {database_name} cascade
# """)

# spark.sql(f"""
# create database if not exists {database_name}
# location 's3://{bucket_name}/{database_name}'
# """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.sql(f"""
use default
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [7]:
# question parameters
spark.sql(f"""drop table if exists {database_name}.questions""")

spark.sql(f"""
create table {database_name}.questions 
using parquet
as
select
    dataset_name,
    post_id,
    post_datetime,
    post_hour,
    post_dayofweek,
    post_month,
    post_year,
    body,
    body_clean,
    body_clean_nocode,
    
    length(body_clean) as body_n_characters,
    length(body_clean_nocode) as body_nocode_n_characters,
    
    1-length(body_clean_nocode)/length(body_clean) as body_code_perc,
    case when body like "%<code>%</code>%" then 1 else 0 end as body_code_flag,
    case when body like "%<img>%</img>%" then 1 else 0 end as body_image_flag,
    case when body like "%<a href%</a>%" then 1 else 0 end as body_link_flag,
    case when body like "%<strong>%</strong>%" or body like "%<b>%</b>%" then 1 else 0 end as body_bold_flag,
    
    title,
    case when title rlike "^[A-Z]" then 1 else 0 end as title_upper_flag,
    case when substr(title, -1, 1) = "?" then 1 else 0 end as title_question_flag,
    length(title) as title_n_characters,
    
    tags,
    n_tags,
    
    post_closed_flag,
    post_score,
    post_score_1d,
    post_score_30d
from (
select 
    p.dataset_name,
    p.id as post_id,
    
    p.creation_date as post_datetime,
    hour(p.creation_date) as post_hour,
    dayofweek(p.creation_date) as post_dayofweek,
    month(p.creation_date) as post_month,
    year(p.creation_date) as post_year,

    p.body,
    trim(regexp_replace(regexp_replace(regexp_replace(p.body, "<.*?>", ""), "\n", " "), " +", " ")) as body_clean,
    trim(regexp_replace(regexp_replace(regexp_replace(regexp_replace(p.body, "(?s)<code>.*<\/code>", ""), "<.*?>", ""), "\n", " "), " +", " ")) as body_clean_nocode,
    p.title,
    p.tags,
    size(p.tags) as n_tags,
    case when p.closed_date is not null then 1 else 0 end as post_closed_flag,
    p.score as post_score,
    count(case when v.vote_type_id = 2 and datediff(v.creation_date, p.creation_date) < 1 then 1 end) -
    count(case when v.vote_type_id = 3 and datediff(v.creation_date, p.creation_date) < 1 then 1 end) as post_score_1d,
    count(case when v.vote_type_id = 2 and datediff(v.creation_date, p.creation_date) < 30 then 1 end) -
    count(case when v.vote_type_id = 3 and datediff(v.creation_date, p.creation_date) < 30 then 1 end) as post_score_30d
    
from posts p
left join votes v
    on v.post_id = p.id
    and p.dataset_name = v.dataset_name
    and v.vote_type_id in (2, 3, 5)
where p.post_type_id = 1
group by
    p.dataset_name,
    p.id,
    p.owner_user_id,
    p.creation_date,
    p.creation_year,
    p.body,
    p.title,
    p.tags,
    p.closed_date,
    p.score
    )
""")

spark.sql(f"""select * from {database_name}.questions""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
# question parameters - NLP
spark.sql(f"""drop table if exists {database_name}.questions_nlp""")

spark.sql(f"""
create table {database_name}.questions_nlp 
using parquet
as
select
    dataset_name,
    post_id,
    udf_count_sentences(body_clean_nocode) as body_nocode_n_sentences,
    udf_count_words(body_clean_nocode) as body_nocode_n_words,
    udf_count_words(title) as title_n_words
    
    /*
    ,
    udf_count_verbs(body_clean_nocode) as body_n_verbs,
    udf_count_nouns(body_clean_nocode) as body_n_nouns,
    udf_count_pronouns(body_clean_nocode) as body_n_prons,
    udf_count_adjectives(body_clean_nocode) as body_n_adjs,
    udf_count_adverbs(body_clean_nocode) as body_n_advs,
    
    udf_get_sentiment_pos(body_clean_nocode) as body_sentiment_pos,
    udf_get_sentiment_neg(body_clean_nocode) as body_sentiment_neg,
    udf_get_sentiment_com(body_clean_nocode) as body_sentiment_compound,
    
    udf_count_verbs(title) as title_n_verbs,
    udf_count_nouns(title) as title_n_nouns,
    udf_count_pronouns(title) as title_n_prons,
    udf_count_adjectives(title) as title_n_adjs,
    udf_count_adverbs(title) as title_n_advs
    */
from (
select 
    p.dataset_name,
    p.id as post_id,
    
    p.body,
    trim(regexp_replace(regexp_replace(regexp_replace(p.body, "<.*?>", ""), "\n", " "), " +", " ")) as body_clean,
    trim(regexp_replace(regexp_replace(regexp_replace(regexp_replace(p.body, "(?s)<code>.*<\/code>", ""), "<.*?>", ""), "\n", " "), " +", " ")) as body_clean_nocode,
    p.title
from posts p
where p.post_type_id = 1
group by
    p.dataset_name,
    p.id,
    p.body,
    p.title
    )
""")

spark.sql(f"""select * from {database_name}.questions_nlp""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------------------------
 dataset_name            | cs-stackexchange-com  
 post_id                 | 62464                 
 body_nocode_n_sentences | 7                     
 body_nocode_n_words     | 113                   
 title_n_words           | 10                    
-RECORD 1----------------------------------------
 dataset_name            | cs-stackexchange-com  
 post_id                 | 64589                 
 body_nocode_n_sentences | 2                     
 body_nocode_n_words     | 88                    
 title_n_words           | 9                     
-RECORD 2----------------------------------------
 dataset_name            | dba-stackexchange-com 
 post_id                 | 127252                
 body_nocode_n_sentences | 2                     
 body_nocode_n_words     | 24                    
 title_n_words           | 4                     
-RECORD 3----------------------------------------
 dataset_name            | dba-stackexchange-com 


In [9]:
# user status
spark.sql(f"""drop table if exists {database_name}.user_status""")

spark.sql(f"""
create table {database_name}.user_status 
using parquet
as
select distinct
    p.dataset_name,
    p.id as post_id,    
    p.owner_user_id as user_id,
    u.display_name as user_display_name,
    
    datediff(p.creation_date, u.creation_date) as user_age_days,
    months_between(p.creation_date, u.creation_date) as user_age_months,
    
    case when u.website_url is not null then 1 else 0 end as user_website_flag,
    case when u.location is not null then 1 else 0 end as user_location_flag,
    case when u.about_me is not null then 1 else 0 end as user_about_me_flag,
    
    count(b.name) as n_badges,
    count(case when b.class = 1 then b.name end) as n_badges_class_1,
    count(case when b.class = 2 then b.name end) as n_badges_class_2,
    count(case when b.class = 3 then b.name end) as n_badges_class_3
from posts p
join users u
    on p.owner_user_id = u.id
    and p.dataset_name = u.dataset_name
left join badges b
    on u.id = b.user_id
    and u.dataset_name = b.dataset_name
    and b.date <= p.creation_date
where p.post_type_id = 1
    and p.owner_user_id is not null
group by
    p.dataset_name,
    p.id,    
    p.owner_user_id,
    u.display_name,
    p.creation_date,
    u.creation_date,
    u.website_url,
    u.location,
    u.about_me
""")

spark.sql(f"""select * from {database_name}.user_status""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------------
 dataset_name       | askubuntu-com                 
 post_id            | 153363                        
 user_id            | 2740                          
 user_display_name  | Sri                           
 user_age_days      | 619                           
 user_age_months    | 20.322671                     
 user_website_flag  | 1                             
 user_location_flag | 1                             
 user_about_me_flag | 1                             
 n_badges           | 5                             
 n_badges_class_1   | 0                             
 n_badges_class_2   | 0                             
 n_badges_class_3   | 5                             
-RECORD 1-------------------------------------------
 dataset_name       | raspberrypi-stackexchange-com 
 post_id            | 37920                         
 user_id            | 8697                          
 user_display_name  | Milliways               

In [10]:
# user history
spark.sql(f"""drop table if exists {database_name}.user_history""")

spark.sql(f"""
create table {database_name}.user_history 
using parquet
as
select 
    p.dataset_name,
    p.id as post_id,    
    p.owner_user_id as user_id,
    
    count(p2.id) as n_user_posts,
    count(case when p2.post_type_id = 1 then p2.id end) as n_user_questions,
    count(case when p2.post_type_id = 2 then p2.id end) as n_user_answers,
    case when count(p2.id) = 0 then 1 else 0 end as user_first_post,
    case when count(case when p2.post_type_id = 1 then p2.id end) = 0 then 1 else 0 end as user_first_question,
    
    count(case when p2.answer_count > 0 then p2.id end) as n_user_answered_questions,
    count(case when p2.accepted_answer_id is not null then p2.id end) as n_user_accepted_answers,
    
    coalesce(sum(p2.score), 0) as user_score,
    coalesce(sum(case when p2.post_type_id = 1 then p2.score end), 0) as user_question_score,
    coalesce(sum(case when p2.post_type_id = 2 then p2.score end), 0) as user_answer_score
from posts p
left join posts p2
    on p.owner_user_id = p2.owner_user_id
    and p.dataset_name = p2.dataset_name
    and p2.creation_date < p.creation_date
where p.post_type_id = 1
group by
    p.dataset_name,
    p.id,    
    p.owner_user_id
""")

spark.sql(f"""select * from {database_name}.user_history""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------------------------------------------
 dataset_name              | devops-stackexchange-com              
 post_id                   | 10                                    
 user_id                   | 15                                    
 n_user_posts              | 0                                     
 n_user_questions          | 0                                     
 n_user_answers            | 0                                     
 user_first_post           | 1                                     
 user_first_question       | 1                                     
 n_user_answered_questions | 0                                     
 n_user_accepted_answers   | 0                                     
 user_score                | 0                                     
 user_question_score       | 0                                     
 user_answer_score         | 0                                     
-RECORD 1---------------------------------------

In [11]:
# tags
spark.sql(f"""drop table if exists {database_name}.tags""")

spark.sql(f"""
create table {database_name}.tags 
using parquet
as
select
    dataset_name, 
    post_id,
    max(n_tag_posts) as n_tag_posts_max,
    max(n_tag_posts_30d) as n_tag_posts_max_30d,
    max(n_tag_posts_365d) as n_tag_posts_max_365d,
    avg(n_tag_posts) as n_tag_posts_avg,
    avg(n_tag_posts_30d) as n_tag_posts_avg_30d,
    avg(n_tag_posts_365d) as n_tag_posts_avg_365d
    from (
select 
    p.dataset_name,
    p.id as post_id,  
    p.tag,
    count(p2.tag) as n_tag_posts,
    count(case when datediff(p.creation_date, p2.creation_date) < 30 then p2.tag end) as n_tag_posts_30d,
    count(case when datediff(p.creation_date, p2.creation_date) < 365 then p2.tag end) as n_tag_posts_365d
from (select p.id, p.dataset_name, p.creation_date, explode(p.tags) as tag from posts p where p.post_type_id = 1) p
left join (select p.id, p.dataset_name, p.creation_date, explode(p.tags) as tag from posts p where p.post_type_id = 1) p2
    on p.tag = p2.tag
    and p.dataset_name = p2.dataset_name
    and p2.creation_date < p.creation_date
group by
    p.dataset_name,
    p.id,  
    p.tag
    )
    group by
        dataset_name, post_id
""")

spark.sql(f"""select * from {database_name}.tags""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------------------------------------------
 dataset_name         | unix-stackexchange-com        
 post_id              | 379935                        
 n_tag_posts_max      | 25                            
 n_tag_posts_max_30d  | 2                             
 n_tag_posts_max_365d | 19                            
 n_tag_posts_avg      | 25.0                          
 n_tag_posts_avg_30d  | 2.0                           
 n_tag_posts_avg_365d | 19.0                          
-RECORD 1---------------------------------------------
 dataset_name         | android-stackexchange-com     
 post_id              | 5017                          
 n_tag_posts_max      | 7                             
 n_tag_posts_max_30d  | 2                             
 n_tag_posts_max_365d | 7                             
 n_tag_posts_avg      | 6.5                           
 n_tag_posts_avg_30d  | 1.5                           
 n_tag_posts_avg_365d | 6.5                           
-RECORD 2-

In [12]:
# forum size
spark.sql(f"""drop table if exists {database_name}.forums""")

spark.sql(f"""
create table {database_name}.forums 
using parquet
as
    select p.dataset_name, 
    year(p.creation_date) as post_year, 
    month(p.creation_date) as post_month,
    count(*) as n_posts,
    count(case when post_type_id = 1 then 1 end) as n_questions,
    count(case when post_type_id = 2 then 1 end) as n_answers
    from posts p
    group by p.dataset_name, year(p.creation_date), month(p.creation_date)
""")

spark.sql(f"""select * from {database_name}.forums order by n_posts desc""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------+----------+-------+-----------+---------+
| dataset_name|post_year|post_month|n_posts|n_questions|n_answers|
+-------------+---------+----------+-------+-----------+---------+
|askubuntu-com|     2012|         5|  11147|       4265|     6642|
|askubuntu-com|     2014|         4|  10430|       4459|     5857|
|askubuntu-com|     2014|         5|  10077|       4261|     5788|
|askubuntu-com|     2013|        10|   9787|       4068|     5613|
|askubuntu-com|     2012|         6|   9721|       3735|     5912|
|askubuntu-com|     2014|         1|   9416|       3707|     5683|
|askubuntu-com|     2012|        10|   9366|       3706|     5554|
|askubuntu-com|     2016|         5|   9312|       4053|     5229|
|askubuntu-com|     2020|         5|   9158|       5278|     3872|
|askubuntu-com|     2012|        11|   9010|       3629|     5327|
|askubuntu-com|     2012|         7|   8779|       3297|     5266|
|askubuntu-com|     2012|         8|   8762|       3329|     5

In [6]:
# answer parameters
spark.sql(f"""drop table if exists {database_name}.answers""")

spark.sql(f"""
create table {database_name}.answers 
using parquet
as
select
    dataset_name,
    post_id,
    user_id,
    count(answer_id) as n_answers,
    count(case when datediff(answer_datetime, post_datetime) < 1 then answer_id end) as n_answers_1d,
    count(case when datediff(answer_datetime, post_datetime) < 30 then answer_id end) as n_answers_30d,
    max(case when answer_id is not null then 1 else 0 end) as answered_flag,
    max(case when datediff(answer_datetime, post_datetime) < 1 then 1 else 0 end) as answered_1d_flag,
    max(case when datediff(answer_datetime, post_datetime) < 7 then 1 else 0 end) as answered_7d_flag,
    max(case when datediff(answer_datetime, post_datetime) < 14 then 1 else 0 end) as answered_14d_flag,
    max(case when datediff(answer_datetime, post_datetime) < 30 then 1 else 0 end) as answered_30d_flag,
    max(answer_score) as answer_max_score,
    max(answer_score_1d) as answer_max_score_1d,
    max(answer_score_30d) as answer_max_score_30d,
    max(answer_accepted_flag) as answer_accepted_flag,
    max(answer_accepted_1d_flag) as answer_accepted_1d_flag,
    max(answer_accepted_7d_flag) as answer_accepted_7d_flag,
    max(answer_accepted_14d_flag) as answer_accepted_14d_flag,
    max(answer_accepted_30d_flag) as answer_accepted_30d_flag
    from (
select 
    p.dataset_name,
    p.id as post_id,    
    p.owner_user_id as user_id,
    p.creation_date as post_datetime,
    
    a.id as answer_id,
    a.creation_date as answer_datetime,
    max(case when va.vote_type_id = 1 then 1 else 0 end) as answer_accepted_flag,
    max(case when va.vote_type_id = 1 and datediff(va.creation_date, p.creation_date) < 1 then 1 else 0 end) as answer_accepted_1d_flag,
    max(case when va.vote_type_id = 1 and datediff(va.creation_date, p.creation_date) < 7 then 1 else 0 end) as answer_accepted_7d_flag,
        max(case when va.vote_type_id = 1 and datediff(va.creation_date, p.creation_date) < 14 then 1 else 0 end) as answer_accepted_14d_flag,
    max(case when va.vote_type_id = 1 and datediff(va.creation_date, p.creation_date) < 30 then 1 else 0 end) as answer_accepted_30d_flag,
    
    coalesce(a.score, 0) as answer_score,
    count(case when va.vote_type_id = 2 and datediff(va.creation_date, p.creation_date) < 1 then 1 end) -
    count(case when va.vote_type_id = 3 and datediff(va.creation_date, p.creation_date) < 1 then 1 end) as answer_score_1d,
    count(case when va.vote_type_id = 2 and datediff(va.creation_date, p.creation_date) < 30 then 1 end) -
    count(case when va.vote_type_id = 3 and datediff(va.creation_date, p.creation_date) < 30 then 1 end) as answer_score_30d
from posts p
left join posts a
    on p.id = a.parent_id
    and p.dataset_name = a.dataset_name
    and a.post_type_id = 2
left join votes va
    on va.post_id = a.id
    and va.dataset_name = a.dataset_name
    and va.vote_type_id in (1, 2, 3)
where p.post_type_id = 1
and ((a.owner_user_id != p.owner_user_id) or a.owner_user_id is null)
group by
    p.dataset_name,
    p.id,
    p.owner_user_id,
    p.creation_date,
    a.id,
    a.creation_date,
    a.score
    ) group by dataset_name, post_id, user_id
""")

spark.sql(f"""select * from {database_name}.answers""").show(5, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------------------
 dataset_name             | unix-stackexchange-com        
 post_id                  | 455013                        
 user_id                  | 50703                         
 n_answers                | 2                             
 n_answers_1d             | 2                             
 n_answers_30d            | 2                             
 answered_flag            | 1                             
 answered_1d_flag         | 1                             
 answered_7d_flag         | 1                             
 answered_14d_flag        | 1                             
 answered_30d_flag        | 1                             
 answer_max_score         | 23                            
 answer_max_score_1d      | 13                            
 answer_max_score_30d     | 16                            
 answer_accepted_flag     | 1                             
 answer_accepted_1d_flag  | 0                           

In [8]:
# final dataset
spark.sql(f"""use {database_name}""")

spark.sql("""
drop table if exists training_data
""")

spark.sql(f"""
create table training_data
using parquet
as
select
    -- features
    q.dataset_name,
    fs.n_posts as dataset_n_posts,
    fs.n_questions as dataset_n_questions,
    fs.n_answers as dataset_n_answers,
    q.post_id,
    q.post_datetime,
    q.post_hour,
    q.post_dayofweek,
    q.post_month,
    q.post_year,
    q.body as post_body,
    q.body_n_characters as post_body_char_count,
    q.body_nocode_n_characters as post_body_nocode_char_count,
    q.body_code_perc as post_body_code_perc,
    q.body_code_flag as post_body_code_flag,
    q.body_image_flag as post_body_image_flag,
    q.body_link_flag as post_body_link_flag,
    q.body_bold_flag as post_body_bold_flag,
    q.title as post_title,
    q.title_upper_flag as post_title_upper_flag,
    q.title_question_flag as post_title_question_flag,
    q.title_n_characters as post_title_char_count,
    q.tags as post_tags,
    q.n_tags as post_tag_count,
    q2.body_nocode_n_sentences as post_body_sent_count,
    q2.body_nocode_n_words as post_body_word_count,
    q2.title_n_words as post_title_word_count,
    t.n_tag_posts_max as tag_post_count_max,
    t.n_tag_posts_max_30d as tag_post_count_30d_max,
    t.n_tag_posts_max_365d as tag_post_count_365d_max,
    t.n_tag_posts_avg as tag_post_count_avg,
    t.n_tag_posts_avg_30d as tag_post_count_30d_avg,
    t.n_tag_posts_avg_365d as tag_post_count_365d_avg,
    us.user_id,
    us.user_age_days,
    us.user_age_months,
    us.user_website_flag,
    us.user_location_flag,
    us.user_about_me_flag,
    us.n_badges as user_badge_count,
    us.n_badges_class_1 as user_badge_1_count,
    us.n_badges_class_2 as user_badge_2_count,
    us.n_badges_class_3 as user_badge_3_count,
    uh.n_user_posts as user_post_count,
    uh.n_user_questions as user_question_count,
    uh.n_user_answers as user_answer_count,
    uh.user_first_post as user_first_post_flag,
    uh.user_first_question as user_first_question_flag,
    uh.n_user_answered_questions as user_answered_questions_count,
    uh.n_user_accepted_answers as user_accepted_answers_count,
    uh.user_score,
    uh.user_question_score,
    uh.user_answer_score,
    -- target
    q.post_closed_flag,
    q.post_score,
    q.post_score_1d,
    q.post_score_30d,
    a.n_answers as answer_count,
    a.n_answers_1d as answer_1d_count,
    a.n_answers_30d as answer_30d_count,
    a.answered_flag as answer_flag,
    a.answered_1d_flag as answer_1d_flag,
    a.answered_7d_flag as answer_7d_flag,
    a.answered_14d_flag as answer_14d_flag,
    a.answered_30d_flag as answer_30d_flag,
    a.answer_max_score,
    a.answer_max_score_1d,
    a.answer_max_score_30d,
    a.answer_accepted_flag,
    a.answer_accepted_1d_flag,
    a.answer_accepted_7d_flag,
    a.answer_accepted_14d_flag,
    a.answer_accepted_30d_flag
from questions q
join questions_nlp q2
    on q.dataset_name = q2.dataset_name
    and q.post_id = q2.post_id
join user_status us
    on q.dataset_name = us.dataset_name
    and q.post_id = us.post_id
join user_history uh
    on q.dataset_name = uh.dataset_name
    and q.post_id = uh.post_id
join tags t
    on q.dataset_name = t.dataset_name
    and q.post_id = t.post_id
join answers a
    on q.dataset_name = a.dataset_name
    and q.post_id = a.post_id
join forums fs
    on q.dataset_name = fs.dataset_name
    and q.post_year = fs.post_year
    and q.post_month = fs.post_month
where q.body_n_characters > 0
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [3]:
spark.sql("""
select * from training_data
""").show(10, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------